<a href="https://colab.research.google.com/github/chrysshawk/Coursera_Capstone/blob/master/ADS_Cap_Wk3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Segmenting and Clustering Neighborhoods in Toronto: Part 1

##Fetch and clean data
In this part, we will scrape Toronto's postal codes from Wikipedia (link  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) and their corresponding geographic coordinates.

First, we start by importing the required libraries for use in this assignment.

In [104]:
# importing necessary libraries
import pandas as pd # library for analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np # library to handle data

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from bs4 import BeautifulSoup # library to process web content
import lxml # libary to parse web content
import urllib.request # import the library we use to open URLs

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Next we will scrape the Wikipedia page for the postal codes by using the urllib for fetching the site, and BeautifulSoup for parsing it.

In [105]:
# Scrape input Wikipedia page
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page = urllib.request.urlopen(wiki_url)
toronto_soup = BeautifulSoup(wiki_page, "lxml") # parsing page

In the webpage, the correct table with Toronto data is defined as a "wikitable sortable" class. We use this information to extract the table information from the toronto soup.

In [106]:
toronto_table = toronto_soup.find('table', class_='wikitable sortable')

However, this table is still in the HTML format, so we will need to extract the data from it into a dataframe.

In [107]:
# First, Initializing lists to pick up the data from the html table
postal_code=[]
borough=[]
neighborhood=[]

# Iterating through the rows of the table and assigning to lists
for row in toronto_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3: # only scrape from entries with 3 colums
        if cells[1].get_text(strip=True) != 'Not assigned': # ignore not assigned boroughs
            postal_code.append(cells[0].get_text(strip=True))
            borough.append(cells[1].get_text(strip=True))
            neighborhood.append(cells[2].get_text(strip=True))

# Creating Toronto dataframe from above lists
toronto_df = pd.DataFrame(postal_code,columns=['PostalCode'])
toronto_df['Borough'] = borough
toronto_df['Neighborhood'] = neighborhood
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


For this project, we will ignore Boroughs that are "Not assigned" from the dataset. To do this, we will drop such rows from the dataframe.

In [108]:
# Dropping Not assigned Boroughs from the dataframe
toronto_df.drop(toronto_df[toronto_df['Borough'] == 'Not assigned'].index, axis=0, inplace=True)

If a cell in the table has a Borough but a "Not assigned" neighborhood, we will assign the neighborhood as the same as the Borough.

In [109]:
# If the Neighborhood is not assigned, while the Borough is, then we set Neighborhood to equal the Borough:
toronto_df.loc[(toronto_df['Neighborhood'] == 'Not assigned') & (toronto_df['Borough'] != 'Not assigned'),
               'Neighborhood'] = toronto_df['Borough']

Let's have a look to see how many entries there are in the table, as well as if there are any duplicates.

In [110]:
print('There are', toronto_df.shape[0], 'postal code entries in the table.')

# Checking for any non-unique Postal Codes
pc_entries = toronto_df['PostalCode'].size
pc_unique = toronto_df['PostalCode'].unique().size

if (pc_entries == pc_unique):
  print('There are no duplicate postal code entries.')
else: print('There are duplicate postal code entries.')

There are 103 postal code entries in the table.
There are no duplicate postal code entries.


As seen above, there are no duplicate postal codes. 

However, in the case there were, we would need to add all the neighborhoods (separated by a comma) into a single postal code, so the end product would be only unique postal codes with potentially multiple neighborhoods within each.

The code for such a task would be as below:

In [111]:
# grouping all neighborhoods within a postal code, and separating multiple entries with comma
neighborhoods_grouped = toronto_df.groupby('PostalCode')['Neighborhood'].transform(lambda x : ', '.join(x))

# assigning the grouped neighborhoods back to original dataframe
toronto_df['Neighborhood'] = neighborhoods_grouped

# removing any duplicate rows (i.e. postal code/neighborhoods) from original
toronto_df.drop_duplicates(inplace=True)

toronto_df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


As the final step in this part, we use the shape command to list the number of rows in the dataframe.

In [112]:
# Printing the number of rows in dataframe:
print('The dataframe has', toronto_df.shape[0], 'rows and', toronto_df.shape[1], 'columns.')

The dataframe has 103 rows and 3 columns.


# Segmenting and Clustering Neighborhoods in Toronto: Part 2
##Retrieving coordinates for postal codes
In this section we will retrieve the coordinates associated with the postal codes.

In [113]:
# Since geocoder package is currently unavailable, we will load the geospatial data provided by Coursera:
geo_url = 'https://cocl.us/Geospatial_data' 
toronto_geo_df = pd.read_csv(geo_url)
print('Geospatial data downloaded and loaded into dataframe. Number of rows:', toronto_geo_df.shape[0])

Geospatial data downloaded and loaded into dataframe. Number of rows: 103


Ok, so it has the same number of rows/postal codes as the dataframe in part 1 above. Let's have a quick look at the dataframe.

In [114]:
toronto_geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now, we will join the geospatial data to the original dataframe from Part 1.

In [115]:
# Left joining geospatial data to postal code in Toronto dataframe
toronto_df = toronto_df.join(toronto_geo_df.set_index('Postal Code'), on='PostalCode', how='left')
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Finally, let us verify there is geospatial data associated with all postal codes.

In [116]:
print('Number of rows missing latitude data: ', toronto_df.loc[toronto_df['Latitude'].isnull()].shape[0])
print('Number of rows missing longitude data: ', toronto_df.loc[toronto_df['Longitude'].isnull()].shape[0])

Number of rows missing latitude data:  0
Number of rows missing longitude data:  0


Zero missing values: This is what we want. Let's move on.

# Toronto, part 3: Clustering neighborhoods in Toronto
In this part we cluster the Toronto neighborhoods using machine learning and K-means clustering. 

First, to make the clustering as meaningful as possible, let's subset the dataset and only look at the Boroughs containing the word "Toronto". That way we can ideally avoid outlier neighborhoods in the overall dataset.

In [117]:
toronto_df = toronto_df.loc[toronto_df['Borough'].str.contains('Toronto')]
print('We new have', toronto_df.shape[0], 'postal codes/neighborhood areas in the dataset.')

We new have 39 postal codes/neighborhood areas in the dataset.


Now, let's plot the current neighborhoods on a map of Toronto.

To begin with, let us find the coordinates of Toronto itself.

In [118]:
# First identifying the location of Toronto, then placing coordinates on a map
address = 'Toronto, Ontario'

# Using the geolocator to identify the geospatial location of Toronto
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


Next, let's plot the neighborhoods from the above dataframe into the map.

In [119]:
# Creating Toronto map using latitude and longitude values as per above
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# Adding neighborhood markers to map
for lat, lng, label in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

# Display map of Toronto with the associated neighborhoods
map_toronto

### Fetching Foursquare data

Having done the above, we need to fetch data from Foursquare in order to get insight into how the various Toronto neighborhoods compare to one another.

First, we need to define the credentials and request parameters we will send to Foursquare.

In [120]:
CLIENT_ID = 'xxx' # your Foursquare ID
CLIENT_SECRET = 'xxx' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit on how many entries Foursquare will return

In [ ]:
#@title
CLIENT_ID = 'MVYIIDNKGNQSTQV5O5KK2H4LZNTUA0OT0XFCMOEKNXZ4BOD2' # your Foursquare ID
CLIENT_SECRET = 'EAFFVEBG41P5BNRGICUCAWFNXWFQ5XWXZOAQKSXS04RRN3YE' # your Foursquare Secret

We intend to carry out this analysis by comparing the various types of venues in the various neighborhoods. To do this efficiently, let's define a function to retrieve the various categories of venues from the results we get from Foursquare.

In [121]:
# Defining function to get the venue categories from Foursquare results
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we need to define a function to make the actuall call to Foursquare.
This will let us retrieve the details of venues from all the Toronto neighborhoods.

In [122]:
# Defining function to retrieve venue details from all Toronto neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL using the Foursquare format and pre-defined paramenters
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    # Creating a dataframe to store the relevant details of the neighborhoods and venues
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Having made the above, we can make call the function with the data of the toronto dataframe we are using for this project.

In [123]:
# Making call to the above function to retrieve Toronto venues
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Ok, having retrieved the data from Foursquare, let us see what the resulting dataframe looks like.

In [124]:
# Having a look at the resulting dataframe:
print('Number of rows:', toronto_venues.shape[0])
print('Number of unique neighborhood areas:', toronto_venues['Neighborhood'].unique().size, '\n')
toronto_venues.head(10)

Number of rows: 1624
Number of unique neighborhood areas: 39 



,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
5,"Regent Park, Harbourfront",43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub
6,"Regent Park, Harbourfront",43.65426,-79.360636,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
7,"Regent Park, Harbourfront",43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park
8,"Regent Park, Harbourfront",43.65426,-79.360636,The Extension Room,43.653313,-79.359725,Gym / Fitness Center
9,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


The above gives us an idea of the general data we are working with.
Additionally, we can see we have venues for 39 unique neighborhood areas, which means we have retrieved venue details on all of the neighborhoods in our scope. If we would not had retrieved venue details for some of the neighborhoods, it would be difficult to compare them to the other neighborhoods.

Next, let's see how many venues we have retrieved per neighborhood.

In [125]:
toronto_venues.groupby('Neighborhood')['Venue'].count()

Neighborhood
Berczy Park                                                                                                    55
Brockton, Parkdale Village, Exhibition Place                                                                   23
Business reply mail Processing Centre, South Central Letter Processing Plant Toronto                           16
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport     16
Central Bay Street                                                                                             68
Christie                                                                                                       16
Church and Wellesley                                                                                           75
Commerce Court, Victoria Hotel                                                                                100
Davisville                                                                 

The above list illustrates an issue: There are some neighborhoods with limited venue data.

For our analysis we will be basing our clustering on the most popular venue categories. However, if some neighborhoods have too little information that becomes impractical.

To make the analysis and clustering of neighborhoods robust, we will therefore remove any neighborhoods with less than 10 different venue categories.

In [126]:
# using map and value count functions to remove Neighborhoods 
# with less than 10 entries:
toronto_venues = toronto_venues[toronto_venues['Neighborhood'].map(
    toronto_venues['Neighborhood'].value_counts()) >= 10]

# resulting neighborhoods:
toronto_venues.groupby('Neighborhood')['Venue'].count()

Neighborhood
Berczy Park                                                                                                    55
Brockton, Parkdale Village, Exhibition Place                                                                   23
Business reply mail Processing Centre, South Central Letter Processing Plant Toronto                           16
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport     16
Central Bay Street                                                                                             68
Christie                                                                                                       16
Church and Wellesley                                                                                           75
Commerce Court, Victoria Hotel                                                                                100
Davisville                                                                 

Finally, let's see how many unique venue categories we are now dealing with.

In [127]:
print('There are {} unique venue categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 unique venue categories.


## Analyzing and clustering each neighborhood

To analyze and cluster each neighborhood, we follow the below steps:
1. Determine the average number of the different venue categories in the neighborhood. These will be the features of the data set.
2. Cluster the neighborhoods based on above features using unsupervised learning and kMeans clustering.
3. Have a look at the different clusters of neighborhoods, evaluate the results of the clustering and analysis. 

### 1. Analyzing the different venue categories in each neighborhood

Here we will analyze the venue categories for each neighborhood. However, first let us check if there are any venue categories labeled "Neighborhood" since these will make it unclear if we are talking about a given neighborhood or a venue category of neighborhood.

In [128]:
# Subsetting on venue category
toronto_venues.loc[toronto_venues['Venue Category']=='Neighborhood']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
413,"Richmond, Adelaide, King",43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Neighborhood
518,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,Harbourfront,43.639526,-79.380688,Neighborhood
958,Studio District,43.659526,-79.340923,Leslieville,43.662070,-79.337856,Neighborhood


So we have 4 neighborhoods of the venue category. To avoid confusion about which type of neighborhood we are talking about, we will rename them as "Neighborhood area".

In [129]:
# Identifying and renaming the Neighborhood category
toronto_venues['Venue Category'] = toronto_venues['Venue Category'].apply(
    lambda x: 'Neighborhood Area' if x == 'Neighborhood' else x)

Next, we will break down the venue categories per neighborhood.

In [130]:
# We break down the various venue categories using one hot encoding:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column as first in dataframe
toronto_onehot = pd.concat([toronto_venues['Neighborhood'], toronto_onehot], axis=1)

Next we will group the neighborhoods and calculate to the mean number of each given venue category in the neighborhood.

In [131]:
# Grouping the neighborhoods by mean number of the various venue categories
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

# Showing the mean occurrence of venue categories per neighborhood
toronto_grouped

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood Area,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,0.00,0.018182,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.0000,0.018182,0.036364,0.000000,0.000000,0.00,0.018182,0.018182,0.000000,0.036364,0.000000,0.000000,0.018182,0.0000,0.000000,0.0000,0.00,0.018182,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.018182,0.00,0.000000,0.0000,0.000000,0.036364,0.000000,0.000000,0.000000,0.000000,0.000000,0.036364,0.090909,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.018182,0.000000,0.018182,0.000000,0.000000,0.000000,0.018182,0.000000,0.00000

Next, we will create a dataframe with the most popular venue categories per neighborhood. To do this, we will first define a function to sort (most popular to least) the venue categories per neighborhood.

In [132]:
# First creating a function to sort the venues descending
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Having done the above, we will create the dataframe to show the 10 most popular venue categories per neighborhood.

In [133]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

# Showing the top rows in the resulting sorted data frame
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Beer Bar,Cocktail Bar,Farmers Market,Seafood Restaurant,Cheese Shop,Restaurant,Sandwich Place,Pub
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Nightclub,Intersection,Stadium,Bakery,Italian Restaurant,Climbing Gym,Restaurant
2,"Business reply mail Processing Centre, South C...",Gym / Fitness Center,Auto Workshop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Pizza Place,Recording Studio
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Plane,Coffee Shop,Sculpture Garden
4,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Salad Place,Department Store,Thai Restaurant,Burger Joint,Bubble Tea Shop,Japanese Restaurant


Ok, we now have prepared data of the various popular venue categories per neighborhood.
With that, we are ready to move on to clustering the various neighborhoods based on their similarities/differences.

### 2. Clustering the neighborhoods according to venue categories
Here we will first set up a K-means clustering algorithm and identify 5 different clusters of neighborhoods.

In [134]:
# set number of clusters
kclusters = 5

# to do the clustering, we need to remove the 'Neighborhood' column since it's not part of the feature set
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 4, 0, 0, 1, 0, 0, 2, 2], dtype=int32)

Finally, we will include the clusters in a dataframe that contains the neighborhood information, the cluster identifyer for the neighborhood, as well as the top venue categories in the neighborhood.

In [135]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_grouped with toronto_df to add latitude/longitude for each neighborhood
# doing right join to only include neighborhoods with associated venue details
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='right')

In [136]:
# Show an example of the data
toronto_merged.sort_values(['Cluster Labels'])

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Bakery,Beer Bar,Cocktail Bar,Farmers Market,Seafood Restaurant,Cheese Shop,Restaurant,Sandwich Place,Pub
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Indian Restaurant,Spa,Caribbean Restaurant
54,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,American Restaurant,Bakery,Brewery,Café,Gastropub,Yoga Studio,Fish Market,Park,Neighborhood Area
92,M5W,Downtown Toronto,Stn A PO Boxes,43.646435,-79.374846,0,Coffee Shop,Italian Restaurant,Seafood Restaurant,Hotel,Beer Bar,Restaurant,Pub,Café,Japanese Restaurant,Cheese Shop
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cocktail Bar,Restaurant,Beer Bar,American Restaurant,Gastropub,Gym,Department Store,Lingerie Store
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,0,Coffee Shop,Café,Gym,Restaurant,Thai Restaurant,Hotel,Bar,Clothing Store,Salad Place,Burrito Place
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Yoga Studio,Event Space,Performing Arts Venue
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Bank,Beer Bar,Smoothie Shop,Sandwich Place,Restaurant,Café,Portuguese Restaurant,Chinese Restaurant,Park
73,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,0,Coffee Shop,Clothing Store,Yoga Studio,Bagel Shop,Furniture / Home Store,Ice Cream Shop,Fast Food Restaurant,Diner,Mexican Restaurant,Chinese Restaurant
37,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,0,Bar,Coffee Shop,Men's Store,Vietnamese Restaurant,Restaurant,Café,Asian Restaurant,Record Shop,Pizza Place,Park


As the final step, we will make a map of the Toronto neighborhoods, where each cluster is identified by a different color.

In [137]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The above summary makes some sense; there is a trend with the central Toronto neighborhoods being in one cluster, while the less central neighborhoods are in a separate cluster.
At the same time, the remaining three clusters seem to be the outlier neighborhoods due to being in more specific locations (e.g. Business reply processing center in cluster 4).

### Conclusion

This concludes the Toronto neighborhood analysis and clustering part of the project. Thank you for evaluating it!